## Question 1a

$1+\frac{1}{1+\frac{1}{1+\frac{1}{1+\frac{1}{1+\frac{1}{1+⋱}}}}}$

## Question 1b
1. [book](book.com)
2. book
3. book
4. book
5. book

## Question 1c
<img href = ""></img>

## Question 1d

## Question 2a

In [43]:
function real_roots_of_quadratic(a::Number, b::Number, c::Number)
    if a == 0
        return "Error - this is not a quadratic equation"
    end
    #Start by initializing an empty array
    roots::Array{Float64} = []

    #Compute the discriminant 
    Δ = b^2 - 4a*c #\Delta + [TAB]

    #Based on the sign of the discriminant return 0, 1, or 2 roots.
    if Δ < 0 
        roots = []
    elseif Δ == 0
        roots = [-b/(2a)]
    else
        roots = (-b .+ [√Δ,-√Δ])/(2a)
    end
    roots 
end

#Attempting on -x²+5x-6=0
real_roots_of_quadratic(0.0,5,-6)

"Error - this is not a quadratic equation"

1. Since Julia is dynamically typed, the type specifications are not necessary and the function will work without them. However, it is often useful to add type annotations for readability and debugging purposes. Specifying types can also be useful if we want to use multiple dispatch to run different methods depending on the type of the parameters specified to a function.
2. `4a*c` is allowed because in Julia variable names cannot start with a number, so Julia recognises the implicit multiplication. However, we cannot write `4ac` because then Julia will try and find a variable named `ac`.
3. If we made a typo and wrote `Δ = 0` instead of `Δ == 0`, then Julia throws a syntax error. However, if we were to put brackets around the condition: `elseif (Δ = 0)`, then the code will run without a syntax error. This is because in this case, Julia treats the assignment as an expression. The expression `Δ = 0` returns the right hand side, `0`. In another language this could lead to a nasty bug because the if statement would never be run, but in Julia conditions must have boolean type, so we would then see a `TypeError`.
4. We can indeed write `(-b + [√Δ,-√Δ])/(2a)`, using the dot operator. If we try the same thing without a dot, then Julia will throw an error, because we cannot add a number to an array. The dot operator applies a function to each element in an array, in this case adding `-b`.
5. The function will still work without the implicit return statement, since by default functions return the value of the last evaluated expression.
6. We cannot use the function with `a = 0.0`, since we will then end up dividing by zero. This will not throw an error, but it means the roots will either be `NaN` or `Inf`, which is clearly not the intended behaviour.
7. At the start of the function we can check if `a==0`, and return a string explaining the error if the function is not quadratic. This is a slightly messy solution.
8. We can shorten the exact same code to
    `a == 0 && return "Error - this is not a quadratic equation"`. This works because Julia has short-circuit evaluation for the and operator. If the first argument is false, then the entire and statement will be false and there is no need to evaluate the other side. If a is 0, then the first expression is true, so Julia needs to check the second expression, which returns the error message.
9. Instead of returning a string, which could lead to problems down the line when using this function, we can explicitly throw an exception with a custom error message: `a == 0 && throw(DomainError(a, "argument must be nonzero"))`. In this case, the most appropriate type of error is a `DomainError`.



## Question 2b

In [35]:
function print_quadratic_roots(coefficients, roots)
    a,b,c = coefficients
    print("The equation $(abs(a) == 1 ? a == 1 ? "" : "-" : a)x² + $(abs(b) == 1 ? b == 1 ? "" : "-" : b)x + $(c) = 0 ") 
    if length(roots) == 0
        println("has no real roots.")
    elseif length(roots) == 1
        println("has the single (real) root $(roots[1])")
    else
        println("has the real roots $(roots[1]) and $(roots[2])")
    end
    return nothing
end

examples = [[1,-5,6], [1,2,3],[1,7,0],[-2,10,12],[-1,-4,+5]]

for example in examples
    roots = real_roots_of_quadratic(example...)
    print_quadratic_roots(example, roots)
end

The equation x² + -5x + 6 = 0 has the real roots 3.0 and 2.0
The equation x² + 2x + 3 = 0 has no real roots.
The equation x² + 7x + 0 = 0 has the real roots 0.0 and -7.0
The equation -2x² + 10x + 12 = 0 has the real roots -1.0 and 6.0
The equation -x² + -4x + 5 = 0 has the real roots -5.0 and 1.0


1. See the examples in the code
2. See code
3. See code

## Question 2c

In [44]:
using Random
"""
This function generates `num_tests` random triples of coefficients and checks that the function `real_roots_of_quadratic()` does its job. The return value is `true` if the test passed, otherwise it is `false`.
"""
function test_real_roots_of_quadratic(;num_tests = 10000, seed=42, coeff_min = -1000, coeff_max = 1000)
    Random.seed!(seed)
    test_passed = true
    for _ in 1:num_tests
        a, b, c = coeff_min .+ (coeff_max-coeff_min)*rand(3) #uniform values in the range [coeff_min, coeff_max]
        roots = real_roots_of_quadratic(a,b,c)
        for x in roots
            err = a*x^2 + b*x + c 
            test_passed = (test_passed && isapprox(err, 0.0, atol = 1e-8)) 
        end
    end
    return test_passed
end

test_real_roots_of_quadratic() ? println("Test passed") : println("Test failed")

Test passed


1. Yes, test still passes
2. See above code
3. If there are no real roots, then the inner for loop will not execute
4. We can add the line `return []` at the top of `real_roots_of_quadratic`, which will mean it always returns an empty array. This means the inner for loop in the testing function will never execute, and the test will pass even though the function is broken.



In [55]:
using Random
function test_from_roots(;num_tests = 10000, seed=42, coeff_min = -1000, coeff_max = 1000)
    Random.seed!(seed)
    test_passed = true
    for _ in 1:num_tests
        num_roots = 1
        # num_roots = rand([0,1,2])
        roots = rand(num_roots)*(coeff_max-coeff_min) .+ coeff_min
        #sort the array so that it doesn't matter what order the function we're testing returns the roots in
        roots = sort(roots)
        a,b,c = 1,1,1
        if num_roots == 0
            a, b, c = rand([(1,0,1),(3,3,3),(-3,-2,-1)]) #pick one of a few hard-coded sets of values
        elseif num_roots == 1
            b = -2*roots[1]
            c = roots[1]^2
        else
            b = -roots[1] - roots[2]
            c = roots[1]*roots[2]
        end
        test_roots = sort(real_roots_of_quadratic(a,b,c))
        #check if the function got the number of roots and the value of the roots right.
        test_passed = length(test_roots) == length(roots) && all(isapprox.(test_roots,roots,atol = 1e-8)) && test_passed
    end
    return test_passed
end

test_from_roots() ? println("Test passed") : println("Test failed")

Test passed


## Question 2d

In [70]:
function roots_of_quadratic(a::Number, b::Number, c::Number)
    roots::Array{ComplexF64} = []
    Δ = b^2 - 4a*c
    roots = [-b + √Complex(Δ), -b - √Complex(Δ)] / (2a)
    return unique([(true ? real(r) : r) for r in roots]) #dedupe and simplify roots
end

roots_of_quadratic(-1,2,3)

2-element Vector{Float64}:
 -1.0
  3.0

## Question 3

In [71]:
function hail_stone_sequence(x::Int, verbose::Bool = false)
    while x != 1
        x = ((x % 2 == 0) ? (x ÷ 2) : (3x + 1)) #\div + [TAB] for ÷ 
        verbose && println("x = $x")
    end
    return nothing
end

hail_stone_sequence (generic function with 2 methods)

## Question 3a

1. The expression `((x % 2 == 0) ? (x ÷ 2) : (3x + 1))` first checks if the remainder is 0 when `x` is divided by 2. i.e. whether `x` is even or not. If so, the expression will return `x ÷ 2` - otherwise (if `x` is odd) it returns `3x + 1`
2. We first define a function $f(n)$ which takes a natural number input $n$. If $n$ is even, then the output is $\frac{n}{2}$. Otherwise the output is $3n+1$. Consider starting with some natural number $m$ and repeatedly applying the function. It is conjecctured that for every choice of $m$, repeatedly applying $f$ will eventually reach an output of 1.
3. See code below (it terminates, so no counterexample)


In [2]:
for n in 1:10^8
    hail_stone_sequence(n)
end

UndefVarError: UndefVarError: hail_stone_sequence not defined

## Question 3c

The given code is inefficient, because the call to `hail_max` involves calculating `hail_length` for all numbers up to `N`. This means that `hail_length` is run multiple times for most numbers, which takes a long time. We can change the code to only calculate `hail_length` once for each number, and store the results in a vector. We can make a further improvement by short-circuiting the calculations if the algorithm reaches a number it has already calculated the sequence for.

The resulting plot up to $10^7$ looks like:
![](plot.png)

The line appears to curve slightly upwards, indicating that the maximum number of steps grows slightly faster than logarithmically. Another interesting feature is that the function does not grow smoothly - there are abrupt leaps where a particular number takes far longer to reach 1 than any number before it.

In [ ]:
using Plots
function hail_length(x::Int, lengths)
    init = x
    n = 0
    while x != 1
        x = ((x % 2 == 0) ? (x ÷ 2) : (3x + 1))
        n += 1
        if x < init #if we have already done these calculations
            n += lengths[x] #look up the remaining length in the list
            break
        end
    end
    return n
end

step_range = 1:10^7
#initialise empty vector lengths
lengths = Array{Int64}(undef, length(step_range))
for i in step_range
    lengths[i] = hail_length(i, lengths)
end

max_vals = accumulate(max,lengths)

plot(step_range, max_vals, xaxis=:log, 
    label=false, xlabel="Maximal initial value", ylabel="Maximal number of steps")

# png("plot.png") #for some reason it doesn't render properly as an svg

## Question 4a

The `random_binary_string` function looks like:

 `random_binary_string(n = 12) = *([rand(['0','1']) for _ in 1:n]...)`

`rand(['0','1'])` will pick a random character, either `'0'` or `'1'`.

We then wrap this in a list comprehension to generate `n` such random numbers: `[rand(['0','1']) for _ in 1:n]`

We then unpack the list using the splatting operator `...`, and concatenate all the strings with the concatenation function `*`. The asterisk could equivalently be replaced with a call to `string`.

The `random_hex_string` function follows a similar approach, escept here the set of characters to choose from is slightly more complex. We want all the characters from 0 to 9, and all the characters from A to F. We can construct this set using the `union` function:

`rand(union('0':'9','A':'F'))`

The above expression therefore picks a random character from the set of hexadecimal symbols. As above, we choose `n` such symbols, and then concatenate them.

## Question 4b

By default `random_binary_string` gives a binary number of length 12 bits. This has a maximum decimal value of $2^12-1 = 4095$ Assuming that our random numbers are uniformly distributed, the expected value is $4095/2 = 2047.5$. Summing over 1000 such values we would expect to see a final sum of around 2047500.

Similarly, `random_hex_string` defaults to a 3-digit hexadecimal number. This has a maximum possible value of $16^3-1 = 4095$, so the expected sum is the same as above.

In [3]:
using Random
random_binary_string(n = 12) = *([rand(['0','1']) for _ in 1:n]...)
random_hex_string(n=3) = *([rand(union('0':'9','A':'F')) for _ in 1:n]...)


Random.seed!(0)
r_bin_strs = [random_binary_string() for _ in 1:10^3]
r_hex_strs = [random_hex_string() for _ in 1:10^3];

bin_nums = parse.(Int,r_bin_strs,base = 2)
hex_nums = parse.(Int,r_hex_strs,base = 16)

sum(bin_nums),sum(hex_nums)

(2001912, 2078359)

## Question 4c

The following custom implementations achieve the same result

In [19]:
function bin2dec(bin_str)
    sum = 0
    for (i,char) in enumerate(reverse(bin_str))
        sum += (char == '0') ? 0 : 2^(i-1)
    end
    return sum
end

function hex2dec(hex_str)
    key = union('0':'9','A':'F')
    sum = 0
    for (i,char) in enumerate(reverse(hex_str))
        sum += (indexin(char,key)[1]-1) * 16^(i-1)
    end
    return sum
end

new_bin_nums = bin2dec.(r_bin_strs)
new_hex_nums = hex2dec.(r_hex_strs)

sum(new_bin_nums),sum(new_hex_nums)

(2001912, 2078359)